# add c_EC as one of the inputs for diffusivity and conductivity; use exponeitial extrapolation for Nyman2008 and make the added function a hat rather than tanh

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp,tanh,sqrt;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import traceback

In [2]:
# define function:
def electrolyte_conductivity_Nyman2008_wEC(c_e,c_EC, T):
    x = c_EC / c_e
    coff = 1
    sigma_e = (
        ((-(0.0+ coff/2 + coff/2 *  tanh((x-4.541*1.5)/1.5))) +
        ( coff/2 + coff/2 *  tanh((x-4.541*0.5)/1.5)) ) * 
        (
        0.1 * 0.06248 * (1+298.15-0.05559) * 
        (c_e/1e3) * (1 - 3.084 *sqrt(c_e/1e3) 
        + 1.33 *(1+ 0.03633 *(exp(1000/298.15))*c_e/1e3)   ) 
        / (1+(c_e/1e3)**4*( 0.00795 *exp(1000/298.15))) )  )
    return sigma_e
def electrolyte_diffusivity_Nyman2008_wEC(c_e, c_EC,T):
    x = c_EC / c_e
    coff = 1
    D_c_e = (
        #((-(0.0+ coff/2 + coff/2 *  tanh((x-4.541*1.5)/1.5))) +
        #( coff/2 + coff/2 *  tanh((x-4.541*0.5)/1.5)) ) * 
        (6 * exp( -1 *(c_e/1000)) 
        * exp(-5/298.15) 
        * exp(-95/298.15*(c_e/1000)) * 1e-10 ) )
    return D_c_e


In [3]:
V_max = 4.2;        V_min = 2.5;
Diff_SEI = 1.7e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 0.3 C until {V_max} V", 
    f"Hold at {V_max} V until C/5")    ] * 1 ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 240 s", )    ] * 1 )   

In [4]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P3R4/'; 
Target  = 'a2_Diff_Cond_c_EC/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);

In [5]:
Model_ddiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "EC wo refill"     } )
Model_sdiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "none"     } )
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);


Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = "[input]" # 1.5e-9/1e2;
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =   1.5e-9/1e2
Para_0['EC transference number'] =     "[input]" # -1.4
Para_0['EC diffusivity in electrolyte [m2.s-1]'] =   "[input]" # 5E-12;
Para_0.update(
    {'Electrolyte diffusivity [m2.s-1]':
    electrolyte_diffusivity_Nyman2008_wEC})
Para_0.update(
    {'Electrolyte conductivity [S.m-1]':
    electrolyte_conductivity_Nyman2008_wEC})
c_e_ddiff = Model_ddiff.variables["Electrolyte concentration [mol.m-3]"]
T_ddiff = Model_ddiff.variables["Cell temperature [K]"]
c_EC_ddiff = Model_ddiff.variables["EC concentration [mol.m-3]"]
Model_ddiff.variables["c(EC) over c(Li+)"] = c_EC_ddiff / c_e_ddiff
Model_ddiff.variables["Electrolyte conductivity [S.m-1]"] =electrolyte_conductivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)
Model_ddiff.variables["Electrolyte diffusivity [m2.s-1]"] =electrolyte_diffusivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)

using EC wo refill for Li+


In [6]:
Sim_ddiff    = pybamm.Simulation(
    Model_ddiff, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)   
Sol_ddiff    = Sim_ddiff.solve(
    inputs={
        "EC transference number":-1.4,
        "Cation transference number":0.28,
        "EC Lithium ion cross diffusivity [m2.s-1]":1.5e-10,
        "EC diffusivity in electrolyte [m2.s-1]":5e-10,});

In [7]:
Model_ddiff.variables.search("potential")

Electrolyte potential
Electrolyte potential [V]
Gradient of electrolyte potential
Gradient of negative electrode potential
Gradient of negative electrolyte potential
Gradient of positive electrode potential
Gradient of positive electrolyte potential
Gradient of separator electrolyte potential
Negative current collector potential
Negative current collector potential [V]
Negative electrode lithium plating reaction overpotential
Negative electrode lithium plating reaction overpotential [V]
Negative electrode open circuit potential
Negative electrode open circuit potential [V]
Negative electrode potential
Negative electrode potential [V]
Negative electrode reaction overpotential
Negative electrode reaction overpotential [V]
Negative electrode surface potential difference
Negative electrode surface potential difference [V]
Negative electrolyte potential
Negative electrolyte potential [V]
Positive current collector potential
Positive current collector potential [V]
Positive electrode open ci

In [8]:
print(Sol_ddiff["Electrolyte conductivity [S.m-1]"].entries[:,51] )
print(Sol_ddiff["Electrolyte diffusivity [m2.s-1]"].entries[:,-1])

[0.16727267 0.17627466 0.19434671 0.22165489 0.25841968 0.30472971
 0.36024401 0.42387894 0.49359678 0.56639703 0.63855565 0.70606933
 0.76517448 0.81276962 0.84659978 0.86514343 0.86722869 0.85146583
 0.81564139 0.75638085 0.71314553 0.70832326 0.70341952 0.69843428
 0.69336755 0.68821942 0.68299001 0.67767948 0.67228808 0.6668161
 0.66126391 0.65563193 0.64992066 0.64413068 0.63826264 0.63231725
 0.62629533 0.62019778 0.61402556 0.60777974 0.5710048  0.50223119
 0.43378272 0.36858154 0.30897693 0.25646094 0.21162699 0.17432537
 0.14391086 0.11948664 0.10009156 0.08481814 0.07287247 0.06359426
 0.05645392 0.05103881 0.04703608 0.04421625 0.04241948 0.04154517]
[1.91045859e-10 1.90882182e-10 1.90554718e-10 1.90063258e-10
 1.89407494e-10 1.88587040e-10 1.87601446e-10 1.86450226e-10
 1.85132891e-10 1.83648984e-10 1.81998132e-10 1.80180104e-10
 1.78194882e-10 1.76042751e-10 1.73724398e-10 1.71241046e-10
 1.68594608e-10 1.65787876e-10 1.62824754e-10 1.59710539e-10
 1.58001279e-10 1.5782325

In [9]:
font = {'family' : 'Times New Roman',
        #'weight' : 'bold',
        'size'   : 18}
mpl.rc('font', **font)

In [10]:
label = ["ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.create_gif(
#    number_of_images=10, duration=2,output_filename=BasicPath + Target+"fixed_add c_EC as tanh hat.gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=4.127123076880398, step=0.041271230768803985…

In [11]:
label = ["Sol_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    ["EC flux [mol.m-2.s-1]","EC flux by diffusion [mol.m-2.s-1]",
        "EC flux by migration [mol.m-2.s-1]","EC flux by Li+ [mol.m-2.s-1]"],
    ["Li+ flux [mol.m-2.s-1]","Li+ flux by diffusion [mol.m-2.s-1]",
        "Li+ flux by migration [mol.m-2.s-1]","Li+ flux by solvent [mol.m-2.s-1]"],    
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff,
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=1,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=4.127123076880398, step=0.041271230768803985…

In [12]:
label = ["Sol_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    "Electrolyte potential [V]",
    "Positive electrode potential [V]",
    "Negative electrode SOC",
    "Positive electrode SOC",
    "Porosity times concentration",
    "eps_c_e rhs",
    "Minus div Li+ flux",
    "Li+ source term",
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff,
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=3,
    figsize = (16,12)) #     spatial_unit='mm',
quick_plot.dynamic_plot();


interactive(children=(FloatSlider(value=0.0, description='t', max=4.127123076880398, step=0.041271230768803985…

In [13]:
label = ["ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff
    ], output_variables3,label,
    variable_limits='tight',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.create_gif(
#    number_of_images=10, duration=2,output_filename=BasicPath + Target+"add c_EC as tanh hat.gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=4.127123076880398, step=0.041271230768803985…